# Poloniex manual spot mm
Places orders at the best bid and best ask of desired symbol
Referral link: https://poloniex.com/signup?c=E6DSDXA6
Referral code: E6DSDXA6

#### Instructions
keep '' put your value between 'valuesgoeshere'
api_key = 'yourkey'
api_secret = 'yoursecret'
Fill in setup with info
spot = 'TRX_USDT'
quantity = '55' this is quantity per side

In [1]:
from polosdk import RestClient
import mplfinance as mpf
import pandas as pd
# import tensorflow as tf
import matplotlib.pyplot as plt 
import mplfinance as mpf
import matplotlib.animation as animation
from matplotlib import style
import datetime as dt
import pytz
import numpy
new_york_timezone = pytz.timezone('America/New_York')
import json

#setup
symbol = 'TRXUSDTPERP'
spot = 'TRX_USDT'
trxbtc = 'TRXBTC'
from polosdk import WsClientPublic
clientoid = 'lilqwantxbt'
make = 'LIMIT_MAKER'
quantity = '55'


def on_message(msg):
    print(msg)

def on_error(err):
    print(err)

import os
import time

api_key = 'put your key'
api_secret = 'put your key'

client = RestClient(api_key, api_secret)
pull_ticker = client.markets().get_ticker24h(symbol=spot)
trxob = client.markets().get_orderbook(spot)
lasttrade = client.markets().get_trades(symbol=spot)
lastprice = lasttrade[0]['price']
mintick = float(trxob['scale'])
bid = float(pull_ticker['bid'])
ask = float(pull_ticker['ask'])
open = float(pull_ticker['open'])
low = float(pull_ticker['low'])
high = float(pull_ticker['high'])
close = float(pull_ticker['close'])
closetime = pull_ticker['closeTime']
markprice = float(pull_ticker['markPrice'])

openorders = client.orders().get_all()

frontbid = markprice - mintick
frontask = markprice + mintick
make = 'LIMIT_MAKER'
# Cancel all orders on 'BTC_USDT'
canall = client.orders().cancel(symbol=spot)


In [11]:
pull_ticker
# Create a limit order for 0.00025 BTC at 20000 USDT
multi_order_request = [
                {
'price': f'{ask}',
'quantity': f'{quantity}',
'side': 'SELL',
'symbol': f'{spot}',
'type': f'{make}',
# 'client_order_id':f'{clientoid}'
                },
                {
'price': f'{bid}',
'quantity': f'{quantity}',
'side': 'BUY',
'symbol': f'{spot}',
'type': f'{make}',
# 'client_order_id':f'{clientoid}'
                }
            ]


# limitsell = client.orders().create(price=lastprice,
#                                   quantity='2',
#                                   side='SELL',
#                                   symbol='WSTUSDT_USDT',
#                                   type=make,
#                                 client_order_id=f'{clientoid}')


multiorder = client.orders().create_multiple(multi_order_request)
multiorder
openorders
ordercount = 0
multioid = []
for orders in openorders:
    multioid.append(orders['id'])

print(multioid)

['266873339283320832']


In [12]:
openorders = client.orders().get_all()
numorders = 0
for thing in openorders:
    numorders +=1
print(numorders)
# openorders
# mylasttrade = client.orders().get_trades(266859578451361793)
# mylasttrade

2


In [13]:
# Cancel all orders on 'BTC_USDT'
canall = client.orders().cancel(symbol=symbol)

In [14]:
!pip freeze > requirements.txt


# dont run anything else after this


In [42]:
transferbal = client.accounts().transfer('USDT', '0.1', 'SPOT', 'FUTURES')
transferbal['transferId']

# Get a list of all accounts of a user with each account’s id, type and balances (assets)
alldetailed = client.accounts().get_balances()
balances = alldetailed[0]['balances']
spotbtc = balances[0]
spotusdt = balances[1]
spottrx = balances[2]
# print(balances)
availxbt = spotbtc['available']
availusdt = spotusdt['available']
availtrx = spottrx['available']
print(availtrx)

# Get fee rate for an account
feerates=client.accounts().get_fee_info()
makerrate = feerates['makerRate']
takerrate = feerates['takerRate']
vol30d = feerates['volume30D']

print(f'30dvol: {vol30d}')

9.543147442497643272
30dvol: 902.379939197949968384


In [26]:
client.get_market(spot)

[{'symbol': 'TRX_USDT',
  'baseCurrencyName': 'TRX',
  'quoteCurrencyName': 'USDT',
  'displayName': 'TRX/USDT',
  'state': 'NORMAL',
  'visibleStartTime': 1659018823984,
  'tradableStartTime': 1659018823985,
  'symbolTradeLimit': {'symbol': 'TRX_USDT',
   'priceScale': 5,
   'quantityScale': 3,
   'amountScale': 5,
   'minQuantity': '0.001',
   'minAmount': '1',
   'highestBid': '0',
   'lowestAsk': '0'},
  'crossMargin': {'supportCrossMargin': True, 'maxLeverage': 3}}]

In [43]:
# Create a market order for 5 USDT of BTC
response = client.orders().create(side='BUY', amount='5', symbol='TRX_USDT')

In [122]:
mylasttrade = client.orders().get_trades()
# mylasttrade[0]

TypeError: get_trades() missing 1 required positional argument: 'order_id'

In [25]:
lasttrade = client.markets().get_trades(symbol='WSTUSDT_USDT')
lastprice = lasttrade[0]['price']

In [ ]:
import asyncio
from polosdk import WsClientPublic

async def on_message(msg):
    global lastprice

    # Check if the message is a 'pong' response
    if msg.get('event') == 'pong':
        print('Received pong')
        return

    # Print the entire message for debugging
    print(msg)

    # Assuming your WebSocket message has a 'data' field containing the actual trade information
    data = msg.get('data', [])

    for trade in data:
        # Assuming your WebSocket message has a 'takerSide' attribute representing the taker side
        lastprice = trade.get('price', '')

        print(lastprice)





async def on_error(err):
    print(err)

async def handle_websocket():
    ws_client_public = WsClientPublic(on_message, on_error=on_error)
    await ws_client_public.connect()
    await ws_client_public.subscribe(['trades'], ['WSTUSDT_USDT'])

    # Simulate running indefinitely to receive WebSocket messages
    try:
        while True:
            await asyncio.sleep(1)
    except asyncio.CancelledError:
        await ws_client_public.disconnect()

# Run the asyncio event loop
await asyncio.create_task(handle_websocket())

# To stop the loop gracefully, use asyncio.gather or other mechanisms
try:
    await asyncio.get_event_loop().run_forever()
except KeyboardInterrupt:
    pass
finally:
    await asyncio.get_event_loop().close()
